Week 3 - Applied Data Science Capstone

# Part 1

Import all require libaries

In [63]:
#!pip install geopy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import geocoder 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab#
#!pip install folium 
import folium 

Scrap the wikipedia page using Beautiful Soup and find the tables on the page.

In [4]:
source = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(source,'html.parser')
tables = soup.find_all('table', class_='sortable')


Iterate through the table to find the individual cells. The cells are saved into lists which are then used to create a pnadas data frame. Any rows that contain an unaasigned borugh are dropped and neighbourhoods are grouped.

In [5]:
postalCode = []
borough = []
neighbourhood = []

for table in tables:
    rows = table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        
        
        if len(cells) > 1:
            field1 = cells[0]
            postalCode.append(field1.text.strip())
            
            field2 = cells[1]
            borough.append(field2.text.strip())
            
            field3 = cells[2]
            neighbourhood.append(field3.text.strip())

df = pd.DataFrame(columns = ['PostalCode','Borough','Neighbourhood'])
df['PostalCode'] = postalCode
df['Borough'] = borough
df['Neighbourhood'] = neighbourhood
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True, inplace = True)
df.groupby('PostalCode')
df.head()


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Unassigend Neighbourhoods are gievn the same value as their Burough.

In [6]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'not Assigned':
        row['Neighbourhood'] = row['Borough']

Output the first 10 rows of the final data frame

In [7]:
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Output the shape of the dataframe

In [8]:
df.shape

(103, 3)

# Part 2

Add two new columns for Latitude and Longitude

In [9]:
df['Latitude'] = ''
df['Longitude'] = ''


Iterate through the rows and add latitude and longitude for each Postal Code.

In [10]:
# initialize your variable to None
lat_lng_coords = None
# loop until you get the coordinates
for index, row in df.iterrows():
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis(row['PostalCode']+', Toronto, Canada')
        lat_lng_coords = g.latlng
    
    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
   

  
df.head(105)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6626,-79.5283
6,M1B,Scarborough,"Malvern, Rouge",43.8114,-79.1966
7,M3B,North York,Don Mills,43.7492,-79.3619
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7072,-79.3119
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378


# Part 3

Make a new dataframe with only the postal codes in Donwtown Toronto

In [11]:
df_dt = df[df.Borough == 'Downtown Toronto']
df_dt.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378
15,M5C,Downtown Toronto,St. James Town,43.6522,-79.3759
20,M5E,Downtown Toronto,Berczy Park,43.6454,-79.3731


Get the latitude and longitude of Tornnto for mapping

In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="data_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Show the Postal Codes areas for Downtown Toronto on a map

In [23]:
map_dt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_dt['Latitude'], df_dt['Longitude'], df_dt['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

Load FourSquare creditentials

In [65]:
CLIENT_ID = '##' # your Foursquare ID
CLIENT_SECRET = 'J##' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ##
CLIENT_SECRET:J##


Define function to get category type

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Define function to get nearby venues

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PC Latitude', 
                  'PC Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get nearby Venues for all Post Codes

In [27]:
Downtown_venues = getNearbyVenues(names=df_dt['PostalCode'],
                                   latitudes=df_dt['Latitude'],
                                   longitudes=df_dt['Longitude']
                                  )

M5A
M7A
M5B
M5C
M5E
M5G
M6G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M5W
M4X
M5X
M4Y


Show head and shape of all venues

In [28]:
print(Downtown_venues.shape)
Downtown_venues.head()

(1244, 7)


,PostalCode,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,M5A,43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,M5A,43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


Group Venues by Postal Code

In [31]:
Downtown_venues.groupby('PostalCode').count()

,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4W,5,5,5,5,5,5
M4X,48,48,48,48,48,48
M4Y,79,79,79,79,79,79
M5A,20,20,20,20,20,20
M5B,100,100,100,100,100,100
M5C,82,82,82,82,82,82
M5E,60,60,60,60,60,60
M5G,76,76,76,76,76,76
M5H,100,100,100,100,100,100


Perfrom One Hot Encoding on data

In [33]:
dt_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['PostalCode'] = Downtown_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()



,PostalCode,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Group One hot encoded data by Postal Code and get Mean

In [34]:
dt_grouped = dt_onehot.groupby('PostalCode').mean().reset_index()
dt_grouped.head()


,PostalCode,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4W,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.200000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.2,0.00,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.2,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.000000
1,M4X,0.000000,0.000000,0.0,0.0,0.00,0.

Show top 5 venue types per postal code

In [36]:
num_top_venues = 5

for pc in dt_grouped['PostalCode']:
    print("----"+pc+"----")
    temp = dt_grouped[dt_grouped['PostalCode'] == pc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
            venue  freq
0      Playground   0.2
1            Park   0.2
2      Bike Trail   0.2
3  Shop & Service   0.2
4    Tennis Court   0.2


----M4X----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2   Restaurant  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----M4Y----
                 venue  freq
0          Coffee Shop  0.11
1              Gay Bar  0.05
2           Restaurant  0.05
3  Japanese Restaurant  0.05
4     Sushi Restaurant  0.04


----M5A----
                  venue  freq
0           Coffee Shop  0.25
1        Breakfast Spot  0.10
2           Yoga Studio  0.05
3               Theater  0.05
4  Gym / Fitness Center  0.05


----M5B----
                       venue  freq
0                Coffee Shop  0.12
1             Clothing Store  0.07
2                       Café  0.04
3        Japanese Restaurant  0.03
4  Middle Eastern Restaurant  0.03


----M5C----
            venue  freq
0     Coffee Shop  0.06
1    Cocktail Bar  0.05
2           

Define function to return most common venue type

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Get top 10 venue sper postal code

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = dt_grouped['PostalCode']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted.head()




,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Playground,Tennis Court,Park,Shop & Service,Bike Trail,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4X,Coffee Shop,Restaurant,Pizza Place,Café,Market,Park,Italian Restaurant,Pub,Bakery,Chinese Restaurant
2,M4Y,Coffee Shop,Gay Bar,Restaurant,Japanese Restaurant,Sushi Restaurant,Café,Mediterranean Restaurant,Pub,Bubble Tea Shop,Men's Store
3,M5A,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Food Truck
4,M5B,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Italian Restaurant,Bookstore,Fast Food Restaurant


Cluster Postal Codes

In [53]:
kclusters = 8

dt_grouped_clustering = dt_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_merged = df_dt

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

dt_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626,4,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Food Truck
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919,5,Coffee Shop,Sandwich Place,Mediterranean Restaurant,Theater,Café,Park,Falafel Restaurant,Fried Chicken Joint,Gastropub,Burrito Place
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Italian Restaurant,Bookstore,Fast Food Restaurant
15,M5C,Downtown Toronto,St. James Town,43.6522,-79.3759,0,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub,Restaurant,Clothing Store,Hotel,Café,American Restaurant,Beer Bar
20,M5E,Downtown Toronto,Berczy Park,43.6454,-79.3731,2,Coffee Shop,Restaurant,Farmers Market,Seafood Restaurant,Cheese Shop,Cocktail Bar,Breakfast Spot,Beer Bar,Bakery,Movie Theater


Map Clusters

In [54]:
# create map
map_clusters = folium.Map(location=[43.652995, -79.398033], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['PostalCode'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Look at details of Clusters

In [55]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5B,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Italian Restaurant,Bookstore,Fast Food Restaurant
15,M5C,0,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub,Restaurant,Clothing Store,Hotel,Café,American Restaurant,Beer Bar
24,M5G,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Restaurant,Sushi Restaurant,Sandwich Place,Bookstore,Plaza,Café,Hotel


In [56]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,M4W,1,Playground,Tennis Court,Park,Shop & Service,Bike Trail,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [57]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5E,2,Coffee Shop,Restaurant,Farmers Market,Seafood Restaurant,Cheese Shop,Cocktail Bar,Breakfast Spot,Beer Bar,Bakery,Movie Theater
30,M5H,2,Hotel,Coffee Shop,Café,Gym,Japanese Restaurant,American Restaurant,Salad Place,Restaurant,Asian Restaurant,Steakhouse
36,M5J,2,Coffee Shop,Hotel,Plaza,Restaurant,Japanese Restaurant,Park,Boat or Ferry,Deli / Bodega,Aquarium,Vegetarian / Vegan Restaurant
42,M5K,2,Coffee Shop,Hotel,Restaurant,Café,American Restaurant,Seafood Restaurant,Japanese Restaurant,Salad Place,Steakhouse,Italian Restaurant
48,M5L,2,Coffee Shop,Restaurant,Hotel,Café,American Restaurant,Gym,Italian Restaurant,Japanese Restaurant,Beer Bar,Deli / Bodega
92,M5W,2,Coffee Shop,Hotel,Restaurant,Italian Restaurant,Café,Bar,Asian Restaurant,Mediterranean Restaurant,Sushi Restaurant,Steakhouse
97,M5X,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Asian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
99,M4Y,2,Coffee Shop,Gay Bar,Restaurant,Japanese Restaurant,Sushi Restaurant,Café,Mediterranean Restaurant,Pub,Bubble Tea Shop,Men's Store


In [58]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M6G,3,Café,Grocery Store,Playground,Coffee Shop,Candy Store,Italian Restaurant,Athletics & Sports,Baby Store,Fast Food Restaurant,Farmers Market


In [59]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,4,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Food Truck


In [60]:
dt_merged.loc[dt_merged['Cluster Labels'] == 5, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M7A,5,Coffee Shop,Sandwich Place,Mediterranean Restaurant,Theater,Café,Park,Falafel Restaurant,Fried Chicken Joint,Gastropub,Burrito Place


In [61]:
dt_merged.loc[dt_merged['Cluster Labels'] == 6, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,M5S,6,Café,Coffee Shop,Bakery,Bookstore,Gym,Japanese Restaurant,Restaurant,Bar,Yoga Studio,Sandwich Place
87,M5V,6,Italian Restaurant,Coffee Shop,Café,French Restaurant,Bar,Park,Gym / Fitness Center,Lounge,Sandwich Place,Speakeasy
96,M4X,6,Coffee Shop,Restaurant,Pizza Place,Café,Market,Park,Italian Restaurant,Pub,Bakery,Chinese Restaurant


In [62]:
dt_merged.loc[dt_merged['Cluster Labels'] == 7, dt_merged.columns[[0] + list(range(5, dt_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,M5T,7,Café,Mexican Restaurant,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Pizza Place,Park,Burger Joint,Thai Restaurant,Dumpling Restaurant


Observations

Two main clusters in the middle with individual postal codes formng their own clusters as yo move our form the center. Cluster 2 is centered around hotels and restaurants, with cluster 1 being very much an outdoor activity cluster